In [1837]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt 

###1. Läsa in data från en SQL-databas (Load data from an SQL database)

In [1838]:
# Create connection to database file
connection = sqlite3.connect("Köksglädje.db")



In [ ]:
products_query = "SELECT * FROM Products;"
products_df = pd.read_sql(products_query, connection)
products_df

In [ ]:
Stores_query = "SELECT * FROM Stores;"
Stores_df = pd.read_sql(Stores_query, connection)
Stores_df

In [ ]:
Transactions_query = 'SELECT * FROM [Transactions];'  
Transactions_df = pd.read_sql(Transactions_query, connection)
Transactions_df

In [ ]:
Customers_query = "SELECT * FROM Customers;"
Customers_df = pd.read_sql(Customers_query, connection)
Customers_df

In [ ]:

connection = sqlite3.connect("Köksglädje.db")
transaction_details_query = "SELECT * FROM TransactionDetails"
transaction_details_df = pd.read_sql(transaction_details_query, connection)
transaction_details_df

In [ ]:
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql(tables_query, connection)
print(tables_df)

###2. Transformera datan i Pandas, om det behövs (Transform the data in Pandas, if needed)

In [ ]:
# Merging transaction_details with products data
df = pd.merge(products_df, transaction_details_df, on="ProductID")
df

In [ ]:

# Merging transactions with store data
df1 = pd.merge(Transactions_df, Stores_df, on="StoreID")
df1

In [ ]:
# Merging transactions with Customer data
df2 = pd.merge(Transactions_df, Customers_df, on="CustomerID")
print(df2.head())




In [ ]:
# Grouping and analyzing CategoryName by product
query = '''SELECT ProductName, Price FROM Products
WHERE CategoryName = "Köksknivar";'''
df.head(20).plot(kind="barh", x="ProductName", rot=0)
df

In [ ]:
# I did Exploratory Data Analysis (EDA) for a df.
float_columns = df.select_dtypes(include='float')
print(float_columns) 

In [ ]:
# Grouping and analyzing quantity by product and mean
df.groupby("ProductName").Quantity.mean()

In [ ]:
query = '''SELECT Productname, Price FROM Products
WHERE ProductName = "Kockkniv";'''
print(df)

In [ ]:
df . describe()

In [ ]:
query = '''SELECT ProductName, Price FROM Products
WHERE CategoryName = "Köksknivar";'''

df.boxplot(figsize = (15,10))
#plt.show()
df

In [ ]:
# Checking missing values and data types
df . isnull().sum()
df.dtypes

In [ ]:
# Checking unique values in the dataset
df .nunique()

In [1856]:
query = '''SELECT ProductName, Price FROM Products 
WHERE CategoryName = "Köksknivar";'''

In [1857]:
df_box = pd.read_sql(query, connection)

In [ ]:
df_box

In [ ]:
df_box.boxplot(figsize=(15,10))
plt.show()

In [ ]:
fig = plt.Figure()
ax = df.boxplot()
plt.show()

In [ ]:
float_columns = df.select_dtypes(include='float')
print(float_columns)

In [ ]:
object_columns = df.select_dtypes(include='object')
print(object_columns)

In [ ]:
integer_columns = df.select_dtypes(include='integer')
print(integer_columns)

In [ ]:
# Query the database
query = """
    SELECT 
        t.TransactionDate, s.StoreName, s.Location, 
        p.ProductName, p.CategoryName, td.Quantity, 
        td.TotalPrice, p.Price, p.CostPrice,
        (td.TotalPrice - (p.CostPrice * td.Quantity)) AS Profit
    FROM TransactionDetails td
    JOIN Products p ON td.ProductID = p.ProductID
    JOIN Transactions t ON td.TransactionID = t.TransactionID
    JOIN Stores s ON t.StoreID = s.StoreID
"""
sales_data = pd.read_sql_query(query, connection)
# Convert TransactionDate to datetime
sales_data["TransactionDate"] = pd.to_datetime(sales_data["TransactionDate"])
# Total sales by store
sales_by_store = sales_data.groupby("StoreName")["TotalPrice"].sum().sort_values(ascending=False)
print("Total Sales by Store:")
print(sales_by_store)

3. Genomföra en analys av datan i Pandas (Conduct data analysis in Pandas)

In [ ]:
# Total sales by product
sales_by_product = sales_data.groupby("ProductName")["TotalPrice"].sum().sort_values(ascending=False).head(10)
print("Top 10 Products by Sales:")
print(sales_by_product)

# Visualization: total sales by product
sales_by_product.plot(kind="bar", title="sales by product", figsize=(5, 7), color="blue")
plt.ylabel("Total Profit ($)")
plt.show()

# Total profit by category
profit_by_category = sales_data.groupby("CategoryName")["Profit"].sum().sort_values(ascending=False)
print("\nTotal Profit by Category:")
print(profit_by_category)

# Visualization: Profit by category
profit_by_category.plot(kind="bar", title="Profit by Category", figsize=(8, 6), color="green")
plt.ylabel("Total Profit ($)")
plt.show()

In [ ]:
# Total sales by store
sales_by_store = sales_data.groupby("StoreName")["TotalPrice"].sum().sort_values(ascending=False)
print("Total Sales by Store:")
print(sales_by_store)

# Visualization (optional)
sales_by_store.plot(kind="bar", title="Total Sales by Store", figsize=(8, 6), color="brown")
plt.ylabel("Total Sales ($)")
plt.show()

In [ ]:
# Total profit by store
profit_by_store = sales_data.groupby("StoreName")["Profit"].sum().sort_values(ascending=False)
print("\nTotal Profit by Store:")
print(profit_by_store)

# Visualization (optional)
profit_by_store.plot(kind="bar", title="Total Profit by Store", figsize=(10, 8), color="darkblue")
plt.ylabel("Total Profit ($)")
plt.show()

In [ ]:
# Merge Data for Analysis
merged_data = (
    transaction_details_df
    .merge(transactions_df, on="TransactionID")
    .merge(products_df, on="ProductID")
    .merge(stores_df, on="StoreID")
)# Top-Selling Products
top_products = (
    merged_data.groupby('ProductName')['TotalPrice']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)
plt.figure(figsize=(10, 8))
top_products.plot(kind='bar', color='green')
plt.title("Top-Selling Products")
plt.xlabel("Product Name")
plt.ylabel("Total Sales (SEK)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Profitability Analysis
merged_data['Profit'] = (merged_data['PriceAtPurchase'] - merged_data['CostPrice']) * merged_data['Quantity']
profit_by_store = merged_data.groupby('StoreName')['Profit'].sum()

plt.figure(figsize=(10, 6))
profit_by_store.sort_values().plot(kind='barh', color='orange')
plt.title("Profit by Store")
plt.xlabel("Total Profit (SEK)")
plt.ylabel("Store Name")
plt.tight_layout()
plt.show()


In [ ]:

##Perform monthly/weekly trend analysis using TransactionDate.
##Find peak sales periods.
sales_data["TransactionDate"] = pd.to_datetime(sales_data["TransactionDate"])
monthly_sales = sales_data.groupby(sales_data["TransactionDate"].dt.to_period("M"))["TotalPrice"].sum()
#  the peak month (highest sales)
peak_month = monthly_sales.idxmax()
peak_sales_value = monthly_sales.max()

print(f"Peak Sales Month: {peak_month}, Sales Amount: {peak_sales_value}")

# Plotting Monthly Sales Trends
monthly_sales.plot(kind="line", title="Monthly Sales Trends")
plt.axhline(y=peak_sales_value, color='r', linestyle='--', label=f"Peak: {peak_month}")
plt.legend()
plt.show()


In [ ]:
##Are there any seasonal patterns in sales?
# Seasonal trends analysis
sales_data["month"] = sales_data["TransactionDate"].dt.month
monthly_sales = sales_data.groupby("month")["TotalPrice"].sum()
monthly_sales.plot(kind="bar", title="Monthly Sales Trends")
plt.show()

In [ ]:
# Query with the correct table name
transaction_query = """
SELECT transaction_date, transactionID, storeID, customerID
FROM "Transactions"  -- Replace with the correct name
"""

try:
    # Execute the query and load the data into a pandas DataFrame
    transaction = pd.read_sql(transaction_query, connection)
    print("Transaction data retrieved successfully!")
    print(transaction.head())
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
print(df.columns)

In [ ]:
#  Connect to the database
connection = sqlite3.connect("Köksglädje.db")

#  Load data from the database with proper checks
try:
    transactions_df = pd.read_sql("SELECT * FROM Transactions;", connection)
    stores_df = pd.read_sql("SELECT * FROM Stores;", connection)
    transaction_details_df = pd.read_sql("SELECT * FROM TransactionDetails;", connection)
    products_df = pd.read_sql("SELECT * FROM Products;", connection)

    # Verify data loaded correctly
    print("Data Loaded Successfully:")
    print(f"Transactions: {transactions_df.shape}, Stores: {stores_df.shape}, "
          f"TransactionDetails: {transaction_details_df.shape}, Products: {products_df.shape}")

except Exception as e:
    print(f"Error loading data: {e}")
    exit()

#  Merge relevant tables
df1 = pd.merge(transactions_df, stores_df, on="StoreID", how="inner")
df1 = pd.merge(df1, transaction_details_df, on="TransactionID", how="inner")
df1 = pd.merge(df1, products_df, on="ProductID", how="inner")

#  Ensure column consistency
df1.columns = df1.columns.str.strip().str.lower()

#  Ensure 'totalprice' column exists
if 'totalprice' not in df1.columns:
    df1["totalprice"] = df1["price"] * df1["quantity"]

#  Analyze sales by store
sales_by_store = df1.groupby("storename")["totalprice"].sum().sort_values(ascending=False)
# Step 7: Display data and visualization
print("Top 5 Stores by Total Sales:")
print(sales_by_store.head())

# Visualization of total sales by store
plt.figure(figsize=(10, 6))
sales_by_store.plot(kind="bar", color="brown")
plt.title("Total Sales by Store")
plt.ylabel("Total Sales ($)")
plt.xlabel("Store Name")
plt.xticks(rotation=45)
plt.show()

#  Data Summary and Missing Values
print("\nData Overview:")
print(df1.describe())

print("\nMissing Values Count:")
print(df1.isnull().sum())

In [ ]:
##. Data Cleaning Techniques
# Data cleaning is essential to ensure accuracy, consistency, and completeness of the dataset. 
##The following techniques were applied:
#Handling Missing Values:

#  Print available columns
print("Initial Columns:", df.columns.tolist())

#  Clean column names
df.columns = df.columns.str.strip()
print("Cleaned Columns:", df.columns.tolist())

#  Check for missing values
print("Missing Values Count:\n", df.isnull().sum())
df.ffill(inplace=True)  # Forward fill missing values

#  Convert data types
if "TransactionDate" in df.columns:
    df["TransactionDate"] = pd.to_datetime(df["TransactionDate"])
else:
    print("Warning: 'TransactionDate' column not found!")

#  Remove duplicates
df.drop_duplicates(inplace=True)

#  Standardize categorical values if StoreName exists
if "StoreName" in df.columns:
    df["StoreName"] = df["StoreName"].str.strip().str.title()
else:
    print("Column 'StoreName' not found. Skipping standardization.")

#  Detect outliers
import seaborn as sns
import matplotlib.pyplot as plt

if "TotalPrice" in df.columns:
    sns.boxplot(x=df["TotalPrice"])
    plt.title("Outlier Detection in TotalPrice")
    plt.show()
else:
    print("Column 'TotalPrice' not found. Cannot plot outliers.")

In [ ]:
#Customer  Purchase Segmentationanalysis
# Remove spaces from column names
df2.columns = df2.columns.str.strip()

# Print columns to verify
print(df2.columns)

# Correct grouping with verified column names
repeat_customers = df2.groupby("CustomerID")["TransactionID"].count().reset_index()
repeat_customers.columns = ["CustomerID", "PurchaseCount"]

# Categorizing customers
repeat_customers["Segment"] = repeat_customers["PurchaseCount"].apply(lambda x: "Repeat" if x > 1 else "New")

# Filter for only repeat customers
repeat_customers_only = repeat_customers[repeat_customers["Segment"] == "Repeat"]

# Display first few rows
print(repeat_customers_only.head())

# Visualization using seaborn and matplotlib
import matplotlib.pyplot as plt

# Create a bar chart for Repeat customers only
plt.figure(figsize=(10, 6))
plt.bar(repeat_customers_only["CustomerID"], repeat_customers_only["PurchaseCount"], color='lightcoral')
plt.title("Repeat Customers")
plt.xlabel("Customer ID")
plt.ylabel("Number of Purchases")
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability

# Display the plot
plt.tight_layout()
plt.show()



In [ ]:

##2.Interactive Data Exploration:
df.describe()
df.hist(figsize=(10, 6))

In [ ]:

#  Print available columns
print("Initial Columns:", df.columns.tolist())

#  Clean column names by stripping extra spaces
df.columns = df.columns.str.strip()
print("Cleaned Columns:", df.columns.tolist())

#  Check for missing values and fill them using forward fill
print("Missing Values Count:\n", df.isnull().sum())
df.ffill(inplace=True)  # Forward fill missing values

#  Convert 'TransactionDate' column to datetime format if it exists
if "TransactionDate" in df.columns:
    df["TransactionDate"] = pd.to_datetime(df["TransactionDate"])
else:
    print("Warning: 'TransactionDate' column not found!")

#  Remove duplicate rows
df.drop_duplicates(inplace=True)

#  Standardize 'StoreName' values if the column exists
if "StoreName" in df.columns:
    df["StoreName"] = df["StoreName"].str.strip().str.title()
else:
    print("Column 'StoreName' not found. Skipping standardization.")

#  Detect outliers in 'TotalPrice' using matplotlib boxplot
if "TotalPrice" in df.columns:
    plt.figure(figsize=(8, 6))  # Set figure size
    plt.boxplot(df["TotalPrice"])
    plt.title("Outlier Detection in TotalPrice")
    plt.xlabel("TotalPrice")
    plt.grid(True)  # Add grid for better visibility
    plt.show(block=True)  # Ensure the plot is displayed properly
else:
    print("Column 'TotalPrice' not found. Cannot plot outliers.")

#  Analyze top-selling product categories
if "CategoryName" in df.columns and "TotalPrice" in df.columns:
    top_categories = df.groupby("CategoryName")["TotalPrice"].sum().sort_values(ascending=False)
    print("\nTop-selling product categories:")
    print(top_categories.head())  # Show top categories
else:
    print("Columns 'CategoryName' or 'TotalPrice' missing. Cannot analyze top categories.")

#  Analyze monthly sales trends
if "TransactionDate" in df.columns and "TotalPrice" in df.columns:
    monthly_sales = df.groupby(df["TransactionDate"].dt.month)["TotalPrice"].sum()
    plt.figure(figsize=(8, 6))
    plt.plot(monthly_sales.index, monthly_sales.values, marker='o', linestyle='-', color='b')
    plt.title("Monthly Sales Trends")
    plt.xlabel("Month")
    plt.ylabel("Total Sales")
    plt.grid(True)
    plt.show()
    print("\nInsight: Sales tend to increase during holiday seasons, particularly in December.")
else:
    print("Cannot calculate monthly sales trends as 'TransactionDate' or 'TotalPrice' is missing.")

In [ ]:
# Här är hur jag har använt EDA på Customers för att undersöka tabellen Customers
connection = sqlite3.connect("Köksglädje.db")
Customers_query = """select CustomerID, JoinDate, ActiveMember, Approvedtocontact from Customers """
Customers_df = pd.read_sql(Customers_query, connection)
print(Customers_df.head())


In [ ]:
Customers_df.info

In [ ]:
Customers_df.max

In [ ]:
Customers_df.shape

In [ ]:
# Här är hur man analyserar tabellen Customers

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select 
Customers_df_query = """select CustomerID, JoinDate, ActiveMember, Approvedtocontact from Customers """

# Här läser vi in data
try:
    Customers_df= pd.read_sql(Customers_query, connection)
    print( "Data is working")
except Exception as e:
    print("Customers tabell doesn't work:",e)
    raise
finally:
    connection.close()

 # Här är hur vi kollar columnerna 
if "JoinDate" in Customers_df.columns:
    print ("Customers is working")
    print("JoinDate info:")
    print(Customers_df['JoinDate'].info())
    
# Här är hur vi använder pd.to_datetime för att skapa se alla datum

    Customers_df['JoinDate']=pd.to_datetime(Customers_df['JoinDate'],errors='coerce')
   
# Här är hur vi skapar Diagrammen 
    
    fig, ax= plt.subplots(figsize=(10,5))
    ax.set(title='Customers Data Display')
    ax.set_xlabel('JoinDate')
    ax.bar(x=Customers_df['JoinDate'],height=Customers_df['CustomerID'],color='green')

    plt.xticks(rotation=45,ha='right')
    plt.tight_layout()
    plt.show()

# TIllsist Här är hur vi printar ut 
else: 
    print("Data closed")
    print(Customers_df['JoinDate'].head())

In [ ]:
# Här är hur man gör analysering på tabellen Transactions och TransactionsDetails

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select och join 
query = """ SELECT Transactions.TransactionDate, TransactionDetails.TotalPrice
from Transactions
Join  TransactionDetails on Transactions.TransactionID = TransactionDetails.TransactionID """

# Här läser vi in data
try:
    df = pd.read_sql(query, connection)
    print("Transaction on go!")
except Exception as e:
    print ("Data doesn't work:",e)
finally:
    connection.close()
    
# Här är hur vi kollar columnerna    
if "TransactionDate" in df.columns and "TotalPrice" in df.columns:
    df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')
    df['TotalPrice'] = df['TotalPrice'].astype(float)
# Här är hur vi printar ut 
    print("Transaction is working")
    print("TransactionDate info:")
    print(df['TransactionDate'].info())

# Här är hur vi har skapat groupby genom summera och använt result index genom att normalisera och sen printa ut det 
    result = df.groupby(['TransactionDate']).sum()
    result = result.reset_index()
    print(result)
# Här har hur man skapar  Diagrammen 
    fig, ax = plt.subplots(figsize=(10,5))
    ax.set_title('Transactions & Graph')
    ax.set_xlabel('TransactionDate')
    ax.set_ylabel('Totalprice')

    ax.plot(result['TransactionDate'], result['TotalPrice'], color='green')

    plt.xticks(rotation=45,ha='right')
    plt.tight_layout()
    plt.show()
    
# Tillsist här är hur vi printar ut 
    print(df.head())
    print(df.info())

In [ ]:
df

In [ ]:
df.TotalPrice.astype(float)

In [ ]:
result

In [ ]:
# Här är hur man gör analysering på tabellen Stores och TransactionDetails

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select och join
query = """select Stores.StoreName,TransactionDetails.Totalprice
from Stores 
Join TransactionDetails on Stores.StoreID = TransactionDetails.TransactionID """

# Här läser vi in data
try:
    df = pd.read_sql(query, connection)
    print(" Stores on go!")
except Exception as e:
    print ("Data doesn't work:",e)
    raise
finally:
        connection.close()
# Här är hur vi kollar columnerna och sen skapar astype som float
if "StoreName" in df.columns and "TotalPrice" in df.columns:
     df['TotalPrice'] = df['TotalPrice'].astype(float)

# Här är hur vi printar   
     print("Stores is working")
     print("StoreName info:")
     print(df["StoreName"].info())
else:
    print('Data is not working')
    print(df.head())
    
# Här är hur man har skapar Diagrammen 
fig, ax = plt.subplots(figsize=(10,5))
df.plot(
x='StoreName',
y='TotalPrice',
kind='bar',
title='Stores',
ax=ax,
color='green' 
)

plt.xticks(rotation=45, ha= 'right')
plt.yticks()
plt.tight_layout()
plt.show()

# Tillsist här är hur vi printar ut 
print(df.head())
print(df.info())

In [ ]:
df

In [ ]:
# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
ActiveMember_query = """select ActiveMember from Customers"""

# Här läser vi in data
df= pd.read_sql(ActiveMember_query, connection)

# Här är hur man kollar alla null kolumner 
null_columns = df[df['ActiveMember'].isnull()]

# Här är hur man printar 
print(null_columns)

# Här är hur man stänger connection
connection.close()

In [ ]:
# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
transactionDetail_query = """select TotalPrice from TransactionDetails"""

# Här läser vi in data
df= pd.read_sql(transactionDetail_query, connection)

# Här är hur vi kollar Total max price i transactiondetails
max_TotalPrice = df['TotalPrice'].max()

# Här är hur man printar 
print(max_TotalPrice)

# Här är hur man stänger connection
connection.close()

In [ ]:
# Detta är minsta Totalprice i transaction Details

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
transactionDetail_query = """select TotalPrice from TransactionDetails"""

# Här läser vi in data
df= pd.read_sql(transactionDetail_query, connection)

# Här är hur vi kollar Total min price i transactiondetails
min_TotalPrice = df['TotalPrice'].min()

# Här är hur man printar 
print(min_TotalPrice)

# Här är hur man stänger connection
connection.close()

In [ ]:
# Här är hur jag har Transformera tabellen Stores

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
query =" select StoreID, Storename, Location from Stores"

# Här läser vi in data
try:
    df= pd.read_sql(query, connection)
    print( "Data is working")
except:
    print("Data is not working")
else:
    print("Data is on go")
finally:
    print("close connection")
    
# Här är hur vi gör groupby för att kolla location genom att använda max
    result = df.groupby('Location').max()

# Här är hur man printar 
    print(result)

In [ ]:
# Här är hur jag har Transformera tabellen Stores

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
query = " select StoreID, StoreName, Location from Stores"

# Här läser vi in data
try:
    df= pd.read_sql(query, connection)
    print( "Data is working")
except:
    print("Data is not working")
else:
    print("Data is on go")
finally:
    print("close connection")
    
# Här är hur vi gör groupby för att kolla storenames columnen genom att använda max
    result = df.groupby('StoreName').max()
   
# Här är hur man printar 
    print(result)

In [ ]:
# Här är hur jag har Transformera tabellen Transactions

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
query = " select TransactionID, StoreID, CustomerID, Transactiondate from Transactions"

# Här läser vi in data
try:
    df= pd.read_sql(query, connection)
    print( "Data is working")
except:
    print("Data is not working")
else:
    print("Data is on go")
finally:
    print("close connection")
    
# Här är hur vi gör groupby för att kolla storeid columnen genom att använda sum
    result = df.groupby('StoreID').sum()
       
# Här är hur man printar 
    print(result)

In [ ]:
# Här är hur jag har Transformera tabellen Products

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
query = " select ProductID, ProductName, Description, CategoryName, Price, CostPrice from Products"

# Här läser vi in data
try:
    df= pd.read_sql(query, connection)
    print( "Data is working")
except:
    print("Data is not working")
else:
    print("Data is on go")
finally:
    print("close connection")

# Här är hur vi gör groupby för att kolla price columnen genom att använda agg.sum,min,max
    result = df.groupby('Price').agg(['min', 'sum', 'max'])

# Här är hur man printar     
    print(result)

In [ ]:
# Här är hur jag har Transformera tabellen products

# Här är Hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")
# Här är hur man hittar information genom att göra select
query = " select ProductID, ProductName, Description, CategoryName, Price, CostPrice from Products"
# Här läser vi in data
try:
    df= pd.read_sql(query, connection)
    print( "Data is working")
except Exception as e:
    print(e)
else:
    print("Data is on go")
finally:
    print("close connection")
    
# Här är hur vi gör groupby för att kolla productname columnen genom att använda prod
    result = df.groupby('ProductName').prod()
    
# Här är hur man printar    
    print(result)

In [ ]:
# Här är hur jag har Transformera tabellen Customers

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
query = "select customerID, Joindate, ActiveMember, Approvedtocontact from customers"

# Här läser vi in data
try:
    df= pd.read_sql(query, connection)
    print( "Data is working")
except Exception as e:
    print(e)
else:
    print("Data is on go")
finally:
    print("close connection")
    
# Här är hur vi gör groupby för att kolla activemember columnen genom att använda max
    result = df.groupby('ActiveMember').max()
    
# Här är hur man printar   
    print(result)

In [ ]:
# Här är hur jag har Transformera tabellen TransactionDetails

# Här är hur man gör connect med sqlite3 med databasen köksglädje
connection =  sqlite3.connect("Köksglädje.db")

# Här är hur man hittar information genom att göra select
query = "select TransactionDetailID, TransactionID, ProductID,  Quantity, PriceAtPurchase, TotalPrice from TransactionDetails"

# Här läser vi in data
try:
    df= pd.read_sql(query, connection)
    print( "Data is working")
except Exception as e:
    print(e)
else:
    print("Data is on go")
finally:
    print("close connection")
    
# Här är hur vi gör groupby för att kolla Totalprice columnen genom att använda sum
    result = df.groupby('TotalPrice').sum()
    
# Här är hur man printar   
    print(result)